In [2]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

print("Project root set to:", project_root)


Project root set to: f:\projects\marketing


In [3]:
from src.config import get_paths
from src.IO import write_parquet
paths = get_paths(project_root)
paths

Paths(root=WindowsPath('F:/projects/marketing'), raw=WindowsPath('F:/projects/marketing/data/raw'), processed=WindowsPath('F:/projects/marketing/data/processed'))

In [4]:
import pandas as pd

In [5]:
from src.IO import read_csv

orders = read_csv(paths.raw / "olist_orders_dataset.csv")
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


### Orders table: column relevance for marketing budget reallocation

The following columns are required for the analysis:
- `order_id`: primary key, required for joins and aggregation
- `customer_id`: required to link orders to customers
- `order_purchase_timestamp`: required for cohorting, recency, and tenure
- `order_status`: required to interpret nulls and exclude canceled orders if needed

The following columns are **not used** in downstream analysis:
- `order_approved_at`: operational timestamp, not relevant to marketing decisions
- `order_delivered_carrier_date`: logistics-related, no impact on acquisition or retention strategy
- `order_delivered_customer_date`: delivery performance, not used for CLV or budget allocation
- `order_estimated_delivery_date`: planning metadata, not required for this analysis

These columns are retained in raw data but excluded from the analytical dataset to reduce noise and focus on decision-relevant features.


In [6]:
orders.shape

(99441, 8)

In [7]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


### Missing values assessment (orders table)

The following columns contain missing values:

- `order_approved_at`: 160 missing values (~0.16%)
- `order_delivered_carrier_date`: 1,783 missing values (~1.79%)
- `order_delivered_customer_date`: 2,965 missing values (~2.98%)

These missing values represent a small fraction of the dataset and are likely related to
orders that were canceled or not fully processed.  
At this stage, rows with missing values are **retained** to avoid prematurely removing
potentially meaningful business events. Handling of these nulls will be informed by
`order_status` during later analysis.

No missing values are present in:
- `order_id`
- `customer_id`
- `order_status`
- `order_purchase_timestamp`
- `order_estimated_delivery_date`


### Data type assessment

All columns in the orders table are currently stored as `object` dtype.

The following columns represent timestamps and should be converted to `datetime`:

- `order_purchase_timestamp`
- `order_approved_at`
- `order_delivered_carrier_date`
- `order_delivered_customer_date`
- `order_estimated_delivery_date`

Converting these columns to datetime is required to support:
- time-based analysis (cohorting, recency, tenure)
- correct handling of missing values
- reliable downstream CLV calculations

Data type conversion will be performed using `pd.to_datetime` with coercion to handle
any invalid or malformed values safely.


In [8]:
datetime_cols = [
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date",
]

for col in datetime_cols:
    orders[col] = pd.to_datetime(orders[col], errors="coerce")


In [9]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](5), object(3)
memory usage: 6.1+ MB


In [10]:
orders[(orders['order_delivered_customer_date'].isnull()) &
       (orders['order_status'] == 'delivered')]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
3002,2d1e2d5bf4dc7227b3bfebb81328c15f,ec05a6d8558c6455f0cbbd8a420ad34f,delivered,2017-11-28 17:44:07,2017-11-28 17:56:40,2017-11-30 18:12:23,NaT,2017-12-18
20618,f5dd62b788049ad9fc0526e3ad11a097,5e89028e024b381dc84a13a3570decb4,delivered,2018-06-20 06:58:43,2018-06-20 07:19:05,2018-06-25 08:05:00,NaT,2018-07-16
43834,2ebdfc4f15f23b91474edf87475f108e,29f0540231702fda0cfdee0a310f11aa,delivered,2018-07-01 17:05:11,2018-07-01 17:15:12,2018-07-03 13:57:00,NaT,2018-07-30
79263,e69f75a717d64fc5ecdfae42b2e8e086,cfda40ca8dd0a5d486a9635b611b398a,delivered,2018-07-01 22:05:55,2018-07-01 22:15:14,2018-07-03 13:57:00,NaT,2018-07-30
82868,0d3268bad9b086af767785e3f0fc0133,4f1d63d35fb7c8999853b2699f5c7649,delivered,2018-07-01 21:14:02,2018-07-01 21:29:54,2018-07-03 09:28:00,NaT,2018-07-24
92643,2d858f451373b04fb5c984a1cc2defaf,e08caf668d499a6d643dafd7c5cc498a,delivered,2017-05-25 23:22:43,2017-05-25 23:30:16,NaT,NaT,2017-06-23
97647,ab7c89dc1bf4a1ead9d6ec1ec8968a84,dd1b84a7286eb4524d52af4256c0ba24,delivered,2018-06-08 12:09:39,2018-06-08 12:36:39,2018-06-12 14:10:00,NaT,2018-06-26
98038,20edc82cf5400ce95e1afacc25798b31,28c37425f1127d887d7337f284080a0f,delivered,2018-06-27 16:09:12,2018-06-27 16:29:30,2018-07-03 19:26:00,NaT,2018-07-19


### Interpretation of missing delivery dates

Missing values in delivery-related timestamp columns are strongly associated with
orders that were not successfully delivered (e.g. canceled or unavailable orders).

When filtering for `order_status == "delivered"`, only 8 rows contain missing
delivery timestamps, representing a negligible fraction of the dataset.

Conclusion:
- Missing delivery timestamps are treated as valid indicators of incomplete orders
- Rows are retained
- Order inclusion/exclusion will be handled explicitly using `order_status`


In [11]:
orders['order_status'].value_counts()

order_status
delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: count, dtype: int64

### Order status filtering for marketing analysis

Order status distribution shows that the majority of orders are marked as `delivered`,
with a smaller number of orders in `shipped`, `approved`, `canceled`, and other states.

For the purposes of marketing budget reallocation, only orders that represent completed
or near-completed customer experiences are included in behavioral analysis.

Included statuses:
- `delivered`
- `shipped` (treated as delivered due to high likelihood of completion)

Excluded statuses:
- `approved`
- `canceled`
- `unavailable`
- other non-completed states

Rationale:
Marketing and CLV analysis require evidence of actual customer consumption.
Orders that did not reach the customer are excluded to avoid biasing repeat behavior
and lifetime value estimates.


In [12]:
valid_statuses = ["delivered", "shipped"]

orders_valid = orders[orders["order_status"].isin(valid_statuses)].copy()


In [13]:
orders_valid['order_status'].value_counts()

order_status
delivered    96478
shipped       1107
Name: count, dtype: int64

In [14]:
write_parquet(
    orders_valid,
    project_root / "data" / "interim" / "orders_valid.parquet"
)